In [7]:
import os.path as op

###############################################################################
# Let us import hnn_core

import hnn_core
from hnn_core import simulate_dipole, Params, Network

hnn_core_root = op.join(op.dirname(hnn_core.__file__), '..')

###############################################################################
# Then we read the parameters file
params_fname = op.join(hnn_core_root, 'param', 'default.json')
params = Params(params_fname)
print(params)

###############################################################################
# This is a lot of parameters! We can also filter the
# parameters using unix-style wildcard characters
print(params['L2Pyr_soma*'])

###############################################################################
# Now let's simulate the dipole
# You can simulate multiple trials in parallel by using n_jobs > 1
# net = Network(params)
# dpls = simulate_dipole(net, n_jobs=1, n_trials=2)


{
    "Itonic_A_L2Basket": 0.0,
    "Itonic_A_L2Pyr_soma": 0.0,
    "Itonic_A_L5Basket": 0.0,
    "Itonic_A_L5Pyr_soma": 0.0,
    "Itonic_T_L2Basket": -1.0,
    "Itonic_T_L2Pyr_soma": -1.0,
    "Itonic_T_L5Basket": -1.0,
    "Itonic_T_L5Pyr_soma": -1.0,
    "Itonic_t0_L2Basket": 0.0,
    "Itonic_t0_L2Pyr_soma": 0.0,
    "Itonic_t0_L5Basket": 0.0,
    "Itonic_t0_L5Pyr_soma": 0.0,
    "L2Basket_Gauss_A_weight": 0.0,
    "L2Basket_Gauss_mu": 2000.0,
    "L2Basket_Gauss_sigma": 3.6,
    "L2Basket_Pois_A_weight_ampa": 0.0,
    "L2Basket_Pois_A_weight_nmda": 0.0,
    "L2Basket_Pois_lamtha": 0.0,
    "L2Pyr_Gauss_A_weight": 0.0,
    "L2Pyr_Gauss_mu": 2000.0,
    "L2Pyr_Gauss_sigma": 3.6,
    "L2Pyr_Pois_A_weight_ampa": 0.0,
    "L2Pyr_Pois_A_weight_nmda": 0.0,
    "L2Pyr_Pois_lamtha": 0.0,
    "L2Pyr_ampa_e": 0.0,
    "L2Pyr_ampa_tau1": 0.5,
    "L2Pyr_ampa_tau2": 5.0,
    "L2Pyr_apical1_L": 306.0,
    "L2Pyr_apical1_diam": 4.08,
    "L2Pyr_apicaloblique_L": 340.0,
    "L2Pyr_apicaloblique_di

In [60]:
from ipywidgets import AppLayout, Button, Layout

In [83]:
from ipywidgets import Output
import matplotlib.pyplot as plt

log_out = Output(layout={'border': '1px solid black'})
plot_out = Output(layout={'border': '1px solid black'})

def on_button_clicked(b, variables):
    with log_out:
        variables['net'] = Network(params)
        dpls = simulate_dipole(variables['net'], n_jobs=1, n_trials=1)
    with plot_out:
        dpls[0].plot()

        
def update_params(**updates):
    params.update(dict(**updates))
    return params

def update_plot(plot_type):
    plot_out.clear_output()

    if not (plot_type['type'] == 'change' and plot_type['name'] == 'value'):
        return
    
    with plot_out:
        if plot_type['new'] == 'spikes':
            variables['net'].plot_spikes()

In [84]:
from ipywidgets import Button, HBox, VBox

In [85]:
def create_expanded_button(description, button_style):
    return Button(description=description, button_style=button_style, layout=Layout(height='10', width='auto'))

header_button = create_expanded_button('Human Neocortical Neurosolver', 'success')
run_button = create_expanded_button('Run', 'success')

In [86]:
def _get_min(v):
    if v < 0:
        return v * 10
    else:
        return v * 0.1

def _get_max(v):
    if v > 0:
        return v * 10
    else:
        return v * 0.1

In [87]:
from ipywidgets import FloatSlider, Dropdown, interactive_output, interactive
from functools import partial

variables = dict(net=None)
run_button.on_click(partial(on_button_clicked, variables=variables))

sliders = [FloatSlider(
    value=params[d], min=_get_min(params[d]), max=_get_max(params[d]), step=0.1,
    description=d,
    disabled=False, continuous_update=False, orientation='horizontal',
    readout=True, readout_format='.2f') for d in params['L2Pyr_soma*'].keys()]

dropdown = Dropdown(
    options=['input histogram', 'dipole current', 'spikes'],
    value='dipole current',
    description='Plot:',
    disabled=False,
)

interactive_output(update_params, {s.description: s for s in sliders})
interactive(update_plot, plot_type='dipole current')
dropdown.observe(update_plot, 'value')

left_box = VBox(sliders)
footer = HBox([run_button, dropdown])

In [88]:
AppLayout(header=header_button,
          left_sidebar=left_box,
          center=log_out,
          right_sidebar=plot_out,
          footer=footer,
          pane_widths=[1, 1, 1])

AppLayout(children=(Button(button_style='success', description='Human Neocortical Neurosolver', layout=Layout(…